In [ ]:
import os
import datetime
from dotenv import load_dotenv
import pandas as pd
import altair as alt

In [ ]:
pd.options.display.max_rows = 50

In [ ]:
WIDTH = 650

In [ ]:
from IPython.display import Markdown
from IPython.core.magic import register_cell_magic

@register_cell_magic
def markdown(line, cell):
    return Markdown(cell.format(**globals()))

In [ ]:
load_dotenv()
GH_TOKEN = os.getenv("GH_TOKEN")

# Living with Machines GitHub Statistics report
This report provides some high level statistics about GitHub repositories contained under the [Living with Machines Organization](https://github.com/Living-with-machines). The report is mainly intended to provide some stats to aid in reporting to our funder.

### Contents
* [Report details](#report-details)
* [Report statistics](#repo-stats)
    * [Overview of repositories](#repo-overview)
    * [Repository file content](#repo-content)
    * [Stars and Forks](#repo-stars)
    * [Views](#repo-views)
    * [Clones](#repo-clones)

## Report details <a class="anchor" id="report-details"></a>

In [ ]:
Markdown(f"This report is updated automatically every week, last generated on: **{datetime.date.today()}**")

In [ ]:
from gh_orgstats.stats import *

In [ ]:
lwm_stats = OrgStats(GH_TOKEN, "Living-with-machines")

In [ ]:
public_names = [repo.name for repo in lwm_stats.public_repos]

# Repository Statistics <a class="anchor" id="repo-stats"></a>

The rest of this document outlines some high level stats for GitHub repositories under the Living with Machines [GitHub Organization](github.com/living-with-machines).

## Overview of repositories <a class="anchor" id="repo-overview"></a>

Currently Living with Machines has the following public repositories: 

In [ ]:
df = pd.DataFrame(public_names,columns=['Repositories']); df.index +=1 
df

In [ ]:
%%markdown
Additionaly Living with Machines has {lwm_stats.private_repo_count} private repositories which have not yet been published

## Repository file content <a class="anchor" id="repo-content"></a>
This section provides an overview of what type of content is in Living with Machines repositories by looking at the file extension counts **note that these counts are based only on default branches so will under count for private repositories**

In [ ]:
df = pd.DataFrame.from_dict(lwm_stats.get_org_file_ext_frequency("public"))
df_private = pd.DataFrame.from_dict(lwm_stats.get_org_file_ext_frequency("private"))

In [ ]:
public_notebook_count = round(df.loc['.ipynb'].sum())
private_notebook_count = round(df_private.loc['.ipynb'].sum())

### Number of Jupyter notebooks
Living with Machines has promised a particular focus on making methods available to other researchers, one way in which the project is aiming to this is through the production of Jupyter notebooks. 

In [ ]:
%%markdown
Living with Machines currently has {public_notebook_count} Jupyter notebooks in public repositories and {private_notebook_count} in private repositories.

### File extensions counts by repository

In [ ]:
df = pd.DataFrame.from_dict(lwm_stats.get_org_file_ext_frequency("public"))
df = df.drop(labels=['.jpg','.json'])
df.reset_index(inplace=True)
data = pd.melt(frame=df, id_vars=["index"], value_name="count")
chart = (
    alt.Chart(data)
    .mark_bar()
    .encode(
        y=alt.Y(
            "index:N",
            title=None,
            sort=alt.EncodingSortField(field="count", order="descending", op="sum"),
        ),
        x=alt.X("count:Q"),
        color=alt.Color("variable:N", title="Repository"),
        tooltip=[alt.Tooltip("variable:N", title="Repository"), "count:Q"],
    )
    .properties(width=WIDTH, title='File Extension Counts')
)
chart

### File extensions by repository

This chart shows the number and type of files for each public Living with Machines repository **note** that this only considers files on the default branch of the repository. We also don't include `.jpg` files or `.json` files since these are often used as for storing data. 

In [ ]:
df = pd.DataFrame.from_dict(lwm_stats.get_org_file_ext_frequency("public"))
df = df.drop(labels=['.jpg','.json'])
df.reset_index(inplace=True)
data = pd.melt(frame=df, id_vars=["index"], value_name="count")
chart = (
    alt.Chart(data)
    .mark_bar()
    .encode(
        y=alt.Y(
            "variable:N",
            title=None,
            sort=alt.EncodingSortField(field="count", order="descending", op="sum"),
        ),
        x=alt.X("count:Q", title="File counts", axis=alt.Axis(tickRound=False)),
        color=alt.Color("index:N", title="File extension"),
        tooltip=[alt.Tooltip("index:N", title="file extension"), "count:Q"],
    )
    .properties(width=WIDTH, title='File extensions by Repository')
)
chart

## Stars and Forks <a class="anchor" id="repo-stars"></a>

[Stars](https://docs.github.com/en/free-pro-team@latest/github/getting-started-with-github/saving-repositories-with-stars) are a GitHub feature which allows registered GitHub users to 'bookmark' a repository. A GitHub [fork](https://docs.github.com/en/free-pro-team@latest/github/getting-started-with-github/fork-a-repo) indicates that someone has made a copy of the repository into their own account. Since stars and forks are only relevant for public repos we only report these below

In [ ]:
df = pd.DataFrame.from_dict(lwm_stats.get_org_snapshot_stats(lwm_stats.public_repos), orient='index')
df.loc['Total']= df.sum(numeric_only=True, axis=0)
df

## Views <a class="anchor" id="repo-views"></a>

GitHub provides owners of a repository with some traffic statistics, including view stats. These are broken into 'unique' and 'total' views.

### Total Views

In [ ]:
df = lwm_stats.get_org_views_traffic(public_only=True, load=True)

In [ ]:
idx = pd.IndexSlice
df = df.resample("W").mean()
total = df.loc[idx[:], idx[:,'total_views']].sum().sum()
mean = df.loc[idx[:], idx[:,'total_views']].sum().mean()
Markdown(f"Living with Machines public repositories have generated {round(total)} total views to date with an average of {round(mean)} daily views")

In [ ]:
data = pd.melt(frame=df.reset_index(), id_vars=['index'],value_name='count')

In [ ]:
brush = alt.selection(type="interval")
chart = (
    alt.Chart(data)
    .mark_area()
    .encode(
        x=alt.X("index:T", axis=alt.Axis(tickCount=24, title="Date")),
        y="sum(count):Q",
        color=alt.Color("variable_0:N", title="Repository"),
        tooltip="sum(count)",
    )
    .transform_filter(alt.FieldEqualPredicate(field="variable_1", equal="total_views"))
    .properties(width=WIDTH, title="Total views")
    .add_selection(brush)
)

bar = (
    alt.Chart(data)
    .mark_bar()
    .encode(
        y=alt.Y("variable_1:N", title=None),
        x="count:Q",
        color=alt.Color("variable_0:N", title="Repository"),
    )
    .properties(title="View by type")
    .transform_filter(brush)
)

## Views over time

The top chart shows us the total views over time by repository, the bottom histogram breaks this down by view type

In [ ]:
chart & bar

## A breakdown of total views for each public Living with Machines repository:

In [ ]:
df_total = df.loc[idx[:], idx[:,'total_views']].stack()
df_total.index = df_total.index.droplevel(1)
pd.DataFrame(df_total.sum().round(0).astype(int),columns=['total_views'])

### Unique views
Unique views aim to not count the same person visiting a repository multiple times

In [ ]:
idx = pd.IndexSlice

In [ ]:
total = df.loc[idx[:], idx[:,'unique_views']].sum().sum()
mean = df.loc[idx[:], idx[:,'unique_views']].sum().mean()
Markdown(f"Living with Machines public repositories have generated {round(total)} unique views to date with an average of {round(mean)} daily views per repository")

### A breakdown of total unique views for each public Living with Machines repository:

In [ ]:
df = df.loc[idx[:], idx[:,'unique_views']].stack()
df.index = df.index.droplevel(1)
pd.DataFrame(df.sum().round(0).astype(int),columns=['unique_views'])

### Mean views by day of week 

In [ ]:
df = lwm_stats.get_org_views_traffic(public_only=True, load=True)
df = df.resample("2D").mean()
data = pd.melt(frame=df.reset_index(), id_vars=['index'],value_name='count')
alt.Chart(data).mark_bar().encode(
    x="mean(count)",
    y=alt.Y("day(index):O", title="Day"),
    color=alt.Color("variable_1", title="View type"),
).properties(width=WIDTH, title="Mean views by Day")


## Clones <a class="anchor" id="repo-clones"></a>

Clones indicate how often a repository is 'downloaded' from GitHub:
> [Cloning a repository pulls down a full copy of all the repository data that GitHub has at that point in time, including all versions of every file and folder for the project.](https://docs.github.com/en/free-pro-team@latest/github/creating-cloning-and-archiving-repositories/cloning-a-repository)

Clones are on way in which we may also be able to assess whether people are making use of a repository. Like views, clones are also broken down into unique and total values. 

### Total Clones

In [ ]:
df = lwm_stats.get_org_clones_traffic(public_only=True, load=True)

In [ ]:
total = df.loc[idx[:], idx[:,'total_clones']].sum().sum()
mean = df.loc[idx[:], idx[:,'total_clones']].sum().mean()
Markdown(f"Living with Machines public repositories have generated {round(total)} clones to date with an average of {round(mean)} daily clones")

## Total clones over time across all repositories 

In [ ]:
df = lwm_stats.get_org_clones_traffic(public_only=True, load=True)
df = df.resample("2D").mean()
data = pd.melt(frame=df.reset_index(), id_vars=['index'],value_name='count')

In [ ]:
brush = alt.selection(type="interval")
chart = (
    alt.Chart(data)
    .mark_area()
    .encode(
        x=alt.X("index:T", axis=alt.Axis(tickCount=24, title="Date")),
        y="sum(count):Q",
        color=alt.Color("variable_0:N", title="Repository"),
     
    )
    .transform_filter(alt.FieldEqualPredicate(field="variable_1", equal="total_clones"))
    .properties(width=WIDTH, title="Total clones")
    .add_selection(brush)
)

bar = (
    alt.Chart(data)
    .mark_bar()
    .encode(
        y=alt.Y("variable_1:N", title=None),
        x="count:Q",
        color=alt.Color("variable_0:N", title="Repository"),tooltip="count:Q"
    )
    .properties(title="Clons by type")
    .transform_filter(brush)
)

chart & bar

### Total clones counts by repository 

In [ ]:
df = lwm_stats.get_org_clones_traffic(public_only=True, load=True)
pd.DataFrame(df.sum().round(0).astype(int),columns=['Clone counts'])

In [ ]:
df = df.resample("2D").mean()

### Mean clones by day of week

In [ ]:
data = pd.melt(frame=df.reset_index(), id_vars=['index'],value_name='count')
alt.Chart(data).mark_bar().encode(
    x="mean(count)",
    y=alt.Y("day(index):O", title="Day"),
    color=alt.Color("variable_1", title="Clone type"),
).properties(width=WIDTH, title="Mean clones by Day")